In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
url = 'https://baseball.yahoo.co.jp/npb/game/2018033001/top'
fetched_dataframes = pd.io.html.read_html(url)

In [3]:
fetched_dataframes[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,1,2,3,4,5,6,7,8,9,計,安,失
1,阪神,0,2,2,0,0,0,1,0,0,5,13,0
2,巨人,0,0,0,0,0,0,1,0,0,1,8,0


In [7]:
result = []
match_id = []
for i in tqdm(range(330, 826)):
    for j in range(6):
        url = 'https://baseball.yahoo.co.jp/npb/game/20180' + str(i) + '0' + str(j+1)+ '/top' 
        try:
            fetched_dataframes = pd.io.html.read_html(url)
            result.append(fetched_dataframes[0])
            match_id.append('20180' + str(i))
        except:
            pass

100%|██████████| 496/496 [25:25<00:00,  3.08s/it]


In [237]:
edit = []

for df in tqdm(result):
    try:
        df.columns = df.iloc[0,:]
        df = df.drop(0).reset_index()
        df['score_at7'] = df.iloc[:, 2:9].sum(axis=1)
        df['flg'] = df['score_at7'][0] < df['score_at7'][1], df['score_at7'][0] > df['score_at7'][1]
        df['result'] = int(df[u'計'][0]) > int(df[u'計'][1]), int(df[u'計'][0]) < int(df[u'計'][1])
        df['reversal'] = df['flg'] & df['result']
        edit.append(df)
    except:
        pass

100%|██████████| 708/708 [00:02<00:00, 324.49it/s]


In [240]:
len(edit)

669

In [241]:
res = edit[0]
for e in edit:
    res = pd.concat([res, e])

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [242]:
res = res.reset_index().drop([0,1]).drop(['level_0', 'index'], axis=1).reset_index()

In [243]:
res['reversal'] = res['flg'] & res['result']

In [246]:
data = res.groupby(res.columns[1])

In [248]:
data.count()[u'index']

nan
オリックス     114
ソフトバンク    108
ヤクルト      109
ロッテ       109
中日        115
全セ          2
全パ          2
巨人        117
広島        110
日本ハム      112
楽天        112
西武        111
阪神        106
ＤｅＮＡ      111
Name: index, dtype: int64

In [254]:
res.head(10)[[res.columns[1], 'reversal']]

,nan,reversal
0,阪神,False
1,巨人,False
2,ヤクルト,False
3,ＤｅＮＡ,False
4,中日,False
5,広島,False
6,西武,False
7,日本ハム,False
8,楽天,True
9,ロッテ,False


In [262]:
fin = data.sum()[['flg', 'reversal']]
fin['rate'] = fin['reversal']/fin['flg']
show = fin.sort_values('rate', ascending=False).drop([u'全パ', u'全セ'])
show.columns = ['七回終了時にリードを許していた試合数', 'うち逆転した試合数', '割合']

In [263]:
show

,七回終了時にリードを許していた試合数,うち逆転した試合数,割合
nan,,,
楽天,55.0,8.0,0.145455
西武,44.0,6.0,0.136364
ソフトバンク,49.0,6.0,0.122449
ヤクルト,52.0,6.0,0.115385
ＤｅＮＡ,64.0,7.0,0.109375
中日,54.0,5.0,0.092593
日本ハム,44.0,4.0,0.090909
オリックス,49.0,4.0,0.081633
広島,38.0,3.0,0.078947
